In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline
import torch
from transformers import TextGenerationPipeline
from tqdm.notebook import tqdm
import os
from dataclasses import dataclass, field
from langchain.chains.base import Chain
from datasets import Dataset
from trl.trainer import ConstantLengthDataset
from peft import LoraConfig
from transformers import DataCollatorForLanguageModeling
from trl.trainer import SFTTrainer, SFTConfig
from transformers import Qwen2ForCausalLM, Qwen2TokenizerFast

In [2]:
os.environ["WANDB_PROJECT"] = "fine-tune-logic-inference"
max_new_tokens = 1024
model_path = "../Qwen2-0.5B"
output_path = "./output"

In [3]:
def format_options(options):
    return '\n'.join(
        [
            f'{chr(ord("A") + i)}: {option}'
            for i, option in enumerate(options)
        ]
    )

In [4]:
@dataclass
class QuestionItem:
    question: str
    options: list[str]
    reasoning: str | None = field(default=None)
    answer: str | None = field(default=None)

@dataclass
class Entry:
    problem: str = field(default="")
    questions: list[QuestionItem] = field(default_factory=list)

In [5]:
def parse_file(file_path: str) -> list[Entry]:
    with open(file_path, "r") as f:
        lines = f.readlines()
    entries = []
    import json
    for line in lines:
        entry = json.loads(line)
        questions = []
        for question in entry["questions"]:
            questions.append(QuestionItem(**question))
        entries.append(Entry(problem=entry["problem"], questions=questions))
    return entries

In [6]:
def reasoning_generation_chain() -> Chain:
    # from transformers import pipeline
    # generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
    from langchain_community.llms.tongyi import Tongyi
    model = Tongyi(model_name="qwen2-7b-instruct")
    from langchain_huggingface import HuggingFacePipeline
    prompt = ChatPromptTemplate.from_messages([
        ("system", '你是一个逻辑推理专家，擅长解决逻辑推理问题。以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都是（close-world assumption）闭世界假设，即未观测事实都为假。每个问题都保证能通过一系列基于形式逻辑的推理（包括同一律，矛盾律，排中律的使用等）得到确定的答案。我会向你提供答案，而你要给出逐步的解析来交会我如何得到答案。我是一个小学生，所以每一步的推理不要太复杂。'),
        ("user", """### 题目:
{problem}

### 问题:
{question}
{options}

### 答案:
{answer}""")])
    return prompt | model

In [7]:
from concurrent.futures import ThreadPoolExecutor

In [8]:
def generate_reasoning_inplace(entries: list[Entry]) -> None:
    chain = reasoning_generation_chain()
    import time
    def process_question(entry, question):
        if question.reasoning is not None:
            return
        max_retries = 3
        for attempt in range(max_retries):
            try:
                reasoning: str = chain.invoke({
                    "problem": entry.problem,
                    "question": question.question,
                    "options": format_options(question.options),
                    "answer": question.answer
                })
                question.reasoning = reasoning
                break  # Exit the loop if successful
            except Exception as e:
                if attempt < max_retries - 1:
                    time.sleep(1)  # Optional: wait a bit before retrying
                else:
                    print(f"Failed to process question.")
                    print(str(e))

    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = []
        for entry in tqdm(entries):
            for question in entry.questions:
                futures.append(executor.submit(process_question, entry, question))
        
        for future in tqdm(futures):
            future.result()

In [9]:
once = False

In [10]:
def create_train_dataset(entries: list[Entry]) -> Dataset:
    dataset = []
    for entry in entries:
        for question in entry.questions:
            if question.reasoning is None or question.answer is None:
                continue
            global once
            if not once:
                print({
                    "problem": entry.problem,
                    "question": question.question,
                    "options": format_options(question.options),
                    "reasoning": question.reasoning,
                    "answer": question.answer
                })
                once = True
            dataset.append({
                "problem": entry.problem,
                "question": question.question,
                "options": format_options(question.options),
                "reasoning": question.reasoning,
                "answer": question.answer
            })
    return Dataset.from_list(dataset)

In [11]:
entries = parse_file("./round1_train_data.jsonl")

In [16]:
generate_reasoning_inplace(entries)

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/1421 [00:00<?, ?it/s]

Failed to process question.
status_code: 400 
 code: DataInspectionFailed 
 message: Output data may contain inappropriate content.
Failed to process question.
status_code: 400 
 code: DataInspectionFailed 
 message: Output data may contain inappropriate content.
Failed to process question.
status_code: 400 
 code: DataInspectionFailed 
 message: Output data may contain inappropriate content.


In [17]:
ds = create_train_dataset(entries)

In [18]:
ds.save_to_disk("./train_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/1418 [00:00<?, ? examples/s]

In [20]:
ds[0]

{'problem': '有一个英文到法文的词汇表，包含以下对应词汇：\n\n1. the -> le\n2. cat -> chat\n3. jumps -> sauts\n4. over -> sur\n5. moon -> lune\n6. cow -> vache\n7. plays -> jouer\n8. fiddle -> violon\n9. egg -> bougre\n10. falls -> des chutes\n11. off -> de\n12. wall -> mur\n\n根据这个词汇表，翻译以下英文句子成法文：',
 'question': '选择题 1：\n英文句子 "the cat jumps over the moon" 翻译成法文是：',
 'options': 'A: le chat saute sur la lune\nB: le chat sauts sur le lune\nC: le sauts chat sur le lune\nD: le chat sauts sur le lune',
 'reasoning': '要解答这个问题，我们首先要根据给定的英文到法文的词汇表逐个翻译句子中的每一个单词。\n\n1. **the** 在法文中对应的词是 **le**。因此，“the” 翻译为 “le”。\n2. **cat** 对应的是 **chat**。\n3. **jumps** 对应的是 **sauts**。注意这里的“sauts”是复数形式，因为原句中的“jumps”也是复数。\n4. **over** 对应的是 **sur**。\n5. **the** 再次出现，对应的仍然是 **le**。\n6. **moon** 对应的是 **lune**。\n\n现在，将这些单词按照英文句子的顺序组合起来，我们得到：\n\n- "the cat jumps over the moon" 翻译为 "le chat sauts sur le lune"\n\n然而，根据选项，我们需要注意到“sauts”是“jumps”的复数形式，所以在法文中，通常会用“sautent”表示复数的跳跃行为。但是，考虑到我们的词汇表中没有提供“sautent”的直接对应，而“sauts”是唯一出现在法文中且与“jumps”对应的复数形式，我

In [15]:
# if __name__ == "__main__":
#     main()